In [0]:
import re
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import plotly
#import graphlab
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='ankit_mishra', api_key='Mbtof2K2TrMllMBUnyok')
plotly.offline.init_notebook_mode()
mapbox_access_token = 'pk.eyJ1IjoiYW5raXRtaXNocmEwNSIsImEiOiJjamhlamNrMDkwazRvMzducWtiYnRiY3R4In0.eLx_c4MJYZZE8NWexKE8cA'
token = 'pk.eyJ1IjoiYW5raXQ0MTgxIiwiYSI6ImNqOGJvaGFlejB3NnIyd3B1ajFzcXRsejIifQ.eGdyw9vf687im_MH7IvzLg' #Replace it with companies token
from mapboxgl.utils import *
from mapboxgl.viz import *

In [0]:
countries_data=pd.read_csv('World_Cities_Location_table.csv',sep=';',header=None)

In [0]:
class PublicSentiment(object):
    column=['Country','City','Positive Sentiment','Negative Sentiment','Neutral Sentiment','Tweets','Lat','Long']
    tweeter_df=pd.DataFrame(columns=column)
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'nqpCbZZPUZLAZnhY2PckAAQY0'
        consumer_secret = 'jVGUQDNAxkZKToBsfPodfRhofeDDWkS3iwF6I3xTb9m2mmm3ku'
        access_token = '995014972932476928-QYMp1kXs2id39kItXT6WN3OxSBDbaRP'
        access_token_secret = 'BYpxPRaMnQJX8yLTApcIebRI0NAB64cRZVeZDvcUEUKDK'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth,wait_on_rate_limit=False,wait_on_rate_limit_notify=True)
        except:
            print("Error: Authentication Failed")
 
    def cleanTweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def getTweetSentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.cleanTweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def getTweets(self,geo, query, count = 100):
        '''
        Main function to fetch tweets and parse them.
        query= the hastag or topic to be searched
        geo= a list containing latitude and longitude; [Lat,Long], max=100
        count= Number of tweets to be fetched
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count,geocode=geo)
            
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.getTweetSentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
    
    def compileModel(self,query):
        #query-the hashtag/topic to be searched(str)
        dataframe=countries_data
        for i in range(100): #evaluating on 243 countries alphabetical order
            print (i)
            loc_lat=dataframe[3][i]#<--pandas dataframe used
            loc_long=dataframe[4][i]#<--pandas dataframe used
            country=dataframe[1][i]
            city=dataframe[2][i]
            tweets = self.getTweets(query = query, count = 100,geo = "%f,%f,%dkm" % (loc_lat, loc_long, 50))
            #print "Picking up Tweets from"+" "+dataframe['Country'][i]+str(len(tweets))
                            # calling function to get tweets
                # picking positive tweets from tweets
            if len(tweets)>10:
                ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
                        # percentage of positive tweets
                    #print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
                        # picking negative tweets from tweets
                ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
                       # percentage of negative tweets
                    #print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
                        # percentage of neutral tweets
                neutral_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']
                    #print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
                posT=100*len(ptweets)/len(tweets)
                negT=100*len(ntweets)/len(tweets)
                neuT=100*len(neutral_tweets)/len(tweets)
                self.tweeter_df.loc[len(self.tweeter_df)] = [country,city,posT,negT,neuT,len(tweets),loc_lat,loc_long]
        #returns the dataframe
        return self.tweeter_df.to_csv('results.csv',sep='\t',encoding='utf-8')